In [1]:
import tensorflow as tf
from tensorflow.keras import models ,layers
#import matplotlib.pyplot as plt
import os
import tensorflow as tf
import keras
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import VGG19
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input,Lambda,Dense,Flatten,Dropout,BatchNormalization,Activation
#from sklearn.metrics import confusion_matrix,classification_report,accuracy_score,recall_score,precision_score,f1_score
#import sys
#import scipy.io
#import scipy.misc
#import matplotlib.pyplot as plt
#from matplotlib.pyplot import imshow
import numpy as np

In [4]:
dataset =tf.keras.preprocessing.image_dataset_from_directory(
    "cl",
    shuffle=True,
    image_size=(256,256),
    batch_size=32,
)

Found 2152 files belonging to 3 classes.


In [5]:
class_names=dataset.class_names
class_names

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

In [6]:
train_size=0.8
l=int(len(dataset)*train_size)
l

54

In [7]:
train_ds=dataset.take(l)
len(train_ds)

54

In [8]:
test_ds=dataset.skip(54)

In [9]:
val_size=0.1
l=int(len(dataset)*val_size)
val_ds=dataset.take(l)

In [10]:
test_ds=dataset.skip(6)

In [11]:
def get_dataset_partition_tf(ds,train_split=0.8,val_split=0.1,test_split=0.1,shuffle=True,shuffle_size=10000):
    ds_size=len(ds)
    if shuffle:
        ds=ds.shuffle(shuffle_size,seed=12)
    train_size=int(ds_size*train_split)
    val_size=int(ds_size*val_split)
    train_ds=ds.take(train_size)
    val_ds=ds.skip(train_size).take(val_size)
    test_ds=ds.skip(train_size).skip(val_size)
    return train_ds,val_ds,test_ds
    

In [12]:
train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds=val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

test_ds=test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

IMAGE_SIZE=256
train_ds,val_ds,test_ds=get_dataset_partition_tf(dataset)

In [13]:
#prefetching the values so that while gpu is traing cpu can load another batch
train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds=val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

test_ds=test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

IMAGE_SIZE=256

In [14]:
resize_and_rescale =tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(IMAGE_SIZE,IMAGE_SIZE),
    layers.experimental.preprocessing.RandomRotation(0.2)
])

In [15]:
data_augmentation = tf.keras.Sequential([
        layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
        layers.experimental.preprocessing.Resizing(IMAGE_SIZE,IMAGE_SIZE),

])

In [16]:
CHANNELS=3
import os
batch_size=32
input_shape=(batch_size,IMAGE_SIZE,IMAGE_SIZE, CHANNELS)
n_classes=3
model=models.Sequential([
    resize_and_rescale,
    data_augmentation,
    layers.Conv2D(32, (3,3),activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, kernel_size= (3,3) , activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, kernel_size= (3,3) , activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dense(n_classes , activation = 'softmax')
])
model.build(input_shape=input_shape)

In [17]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (32, 256, 256, 3)         0         
_________________________________________________________________
sequential_1 (Sequential)    (32, 256, 256, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (32, 254, 254, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (32, 127, 127, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (32, 125, 125, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (32, 62, 62, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (32, 60, 60, 64)         

In [18]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
    

)

In [19]:
EPOCHS=50

In [20]:
history=model.fit(
    train_ds,
    epochs=EPOCHS,
    batch_size=batch_size,
    verbose=1,
    validation_data=val_ds
)

Epoch 1/50
54/54 [==============================] - 18s 98ms/step - loss: 1.7755 - accuracy: 0.4786 - val_loss: 0.8367 - val_accuracy: 0.5573
Epoch 2/50
54/54 [==============================] - 4s 77ms/step - loss: 0.7275 - accuracy: 0.6597 - val_loss: 0.8835 - val_accuracy: 0.5365
Epoch 3/50
54/54 [==============================] - 4s 77ms/step - loss: 0.5273 - accuracy: 0.7627 - val_loss: 0.2968 - val_accuracy: 0.8958
Epoch 4/50
54/54 [==============================] - 4s 76ms/step - loss: 0.2985 - accuracy: 0.8837 - val_loss: 0.2657 - val_accuracy: 0.8750
Epoch 5/50
54/54 [==============================] - 4s 77ms/step - loss: 0.2601 - accuracy: 0.8958 - val_loss: 0.5216 - val_accuracy: 0.8073
Epoch 6/50
54/54 [==============================] - 4s 77ms/step - loss: 0.2516 - accuracy: 0.9086 - val_loss: 0.1792 - val_accuracy: 0.9375
Epoch 7/50
54/54 [==============================] - 4s 77ms/step - loss: 0.1989 - accuracy: 0.9265 - val_loss: 0.1942 - val_accuracy: 0.9219
Epoch 8/50
5

In [21]:
scores=model.evaluate(test_ds)

8/8 [==============================] - 1s 42ms/step - loss: 0.0273 - accuracy: 0.9922


In [22]:
scores

[0.02734120562672615, 0.9921875]

In [23]:
accuracy=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

In [24]:
def predict(model,img):
    img_array=tf.keras.preprocessing.image.img_to_array(images[i].numpy)
    img_array=tf.expand_dims(img_array,0)
    
    predictions= model.predict(img_array)
    predicted_class=class_names[np.argmax(predictions[0])]
    confidence=round(100*(np.max(predictions[0])),2)
    return predicted_class ,confidence

In [27]:
from joblib import dump


In [34]:
model_version=1
model.save(f"./../models/model1")


INFO:tensorflow:Assets written to: ./../models/model1\assets


In [35]:
model.save('m.h5')

In [36]:
import keras
from keras.models import load_model

In [37]:
load_model('m.h5')